In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
import pickle as pkl
import ast
import os
from sklearn.metrics.pairwise import cosine_similarity
from random import sample
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
import pickle as pkl
import ast
import os
from random import sample
import networkx as nx

from random import sample
import sklearn
import re
import string
import warnings
from bs4 import BeautifulSoup
import gensim

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
month = 'july'

framepath = "/data/shruti/ONR/small_data/embeddings/" + month + "_frame.csv"
grouppath = "../lite_data/results/monthly_grouped_texts/" + month + "_grouped_texts.csv"
writepath = "/data/shruti/ONR/small_data/facebook_monthly_text_groups/" + month + ".csv"

In [3]:
sample_frame = pd.read_csv(framepath, header=0)

In [4]:
sample_groups = pd.read_csv(grouppath, header=0)

In [5]:
sample_frame.head()

,Unnamed: 0,date,title,description,link,postUrl,score,handle,platformId,id,message,caption,clean_text,textlen
0,3,2021-07-31 23:57:55,NaN,NaN,NaN,https://www.facebook.com/groups/20430641726032...,-1.296296,"*श्री ""राधा रानी"" प्राण हमारी ""वृंदावन"" मेरो ज...",2043064172603248,8043682,सती विरह में शिव घूमे लेकर देह तीनो लोक उमा पा...,NaN,सती विरह में शिव घूमे लेकर देह तीनो लोक उमा पा...,22
1,4,2021-07-31 23:57:40,NaN,NaN,NaN,https://www.facebook.com/groups/33274896690677...,1.321429,Satsang-सत्संग-पौराणिक कहानियाँ-ध्यान-भक्ति-री...,332748966906779,9060585,सती विरह में शिव घूमे लेकर देह तीनो लोक उमा पा...,NaN,सती विरह में शिव घूमे लेकर देह तीनो लोक उमा पा...,22
2,9,2021-07-31 23:54:58,मनकामना माईको दर्शन गरि आजको राशिफल हेर्नुहोस।...,वि.सं.२०७८ साल साउन १७ गते । आईतबार । ईश्वी सन...,https://khabarpatrikanp.com/archives/22725,https://www.facebook.com/groups/12800605888173...,-3.571429,राजा ज्ञानेन्द्र - King Gyanendra (राष्ट्रभक्त...,1280060588817309,14759682,मनकामना माईको दर्शन गरि आजको राशिफल हेर्नुहोस।...,khabarpatrikanp.com,मनकामना माईको दर्शन गरि आजको राशिफल हेर्नुहोस ...,17
3,10,2021-07-31 23:54:54,"भगवती माताले सबैको मनोकांक्षा पुरा गरुन,श्रावण...","मेष राशि चु, चे, चो, ला, लि, लु, ले, लो, अ साम...",https://www.mbinfomedia.com/archives/3645,https://www.facebook.com/groups/54992145928912...,-1.780488,🇳🇵कृष्ण कडेल A.B.C इन्द्रेणी समर्थक G.C🇳🇵,549921459289121,14658604,"भगवती माताले सबैको मनोकांक्षा पुरा गरुन,श्रावण...",mbinfomedia.com,भगवती माताले सबैको मनोकांक्षा पुरा गरुनश्रावण ...,16
4,16,2021-07-31 23:52:03,"बिहार ,अवैध शराब के खिलाफ कार्यवाई में पुलिस क...",मगध एक्सप्रेस ;-बिहार के मुजफ्फरपुर पुलिस की व...,https://magadhexpress.in/?p=71252,https://www.facebook.com/groups/25065276533930...,-10.333333,औरंगाबाद जिले का नाम देव हो,250652765339303,9063079,"बिहार ,अवैध शराब के खिलाफ कार्यवाई में पुलिस क...",magadhexpress.in,बिहार अवैध शराब के खिलाफ कार्यवाई में पुलिस को...,27


In [6]:
sample_groups.head()

,Id,Label,timeset,componentnumber
0,50188,50188,NaN,0
1,51394,51394,NaN,0
2,15359,15359,NaN,1
3,36816,36816,NaN,1
4,19109,19109,NaN,2


In [7]:
sample_frame['text_no'] = list(range(len(sample_frame)))

In [8]:
sample_frame.head()

,Unnamed: 0,date,title,description,link,postUrl,score,handle,platformId,id,message,caption,clean_text,textlen,text_no
0,3,2021-07-31 23:57:55,NaN,NaN,NaN,https://www.facebook.com/groups/20430641726032...,-1.296296,"*श्री ""राधा रानी"" प्राण हमारी ""वृंदावन"" मेरो ज...",2043064172603248,8043682,सती विरह में शिव घूमे लेकर देह तीनो लोक उमा पा...,NaN,सती विरह में शिव घूमे लेकर देह तीनो लोक उमा पा...,22,0
1,4,2021-07-31 23:57:40,NaN,NaN,NaN,https://www.facebook.com/groups/33274896690677...,1.321429,Satsang-सत्संग-पौराणिक कहानियाँ-ध्यान-भक्ति-री...,332748966906779,9060585,सती विरह में शिव घूमे लेकर देह तीनो लोक उमा पा...,NaN,सती विरह में शिव घूमे लेकर देह तीनो लोक उमा पा...,22,1
2,9,2021-07-31 23:54:58,मनकामना माईको दर्शन गरि आजको राशिफल हेर्नुहोस।...,वि.सं.२०७८ साल साउन १७ गते । आईतबार । ईश्वी सन...,https://khabarpatrikanp.com/archives/22725,https://www.facebook.com/groups/12800605888173...,-3.571429,राजा ज्ञानेन्द्र - King Gyanendra (राष्ट्रभक्त...,1280060588817309,14759682,मनकामना माईको दर्शन गरि आजको राशिफल हेर्नुहोस।...,khabarpatrikanp.com,मनकामना माईको दर्शन गरि आजको राशिफल हेर्नुहोस ...,17,2
3,10,2021-07-31 23:54:54,"भगवती माताले सबैको मनोकांक्षा पुरा गरुन,श्रावण...","मेष राशि चु, चे, चो, ला, लि, लु, ले, लो, अ साम...",https://www.mbinfomedia.com/archives/3645,https://www.facebook.com/groups/54992145928912...,-1.780488,🇳🇵कृष्ण कडेल A.B.C इन्द्रेणी समर्थक G.C🇳🇵,549921459289121,14658604,"भगवती माताले सबैको मनोकांक्षा पुरा गरुन,श्रावण...",mbinfomedia.com,भगवती माताले सबैको मनोकांक्षा पुरा गरुनश्रावण ...,16,3
4,16,2021-07-31 23:52:03,"बिहार ,अवैध शराब के खिलाफ कार्यवाई में पुलिस क...",मगध एक्सप्रेस ;-बिहार के मुजफ्फरपुर पुलिस की व...,https://magadhexpress.in/?p=71252,https://www.facebook.com/groups/25065276533930...,-10.333333,औरंगाबाद जिले का नाम देव हो,250652765339303,9063079,"बिहार ,अवैध शराब के खिलाफ कार्यवाई में पुलिस क...",magadhexpress.in,बिहार अवैध शराब के खिलाफ कार्यवाई में पुलिस को...,27,4


In [9]:
id2component = sample_groups.set_index('Id').to_dict()['componentnumber']
id2component

{50188: 0,
 51394: 0,
 15359: 1,
 36816: 1,
 19109: 2,
 20125: 2,
 31141: 3,
 54927: 3,
 57941: 4,
 92020: 4,
 20846: 5,
 107458: 5,
 40367: 5,
 50625: 5,
 84882: 5,
 98012: 5,
 109542: 6,
 109583: 6,
 134467: 7,
 134469: 7,
 84691: 8,
 86366: 8,
 63044: 1,
 127209: 1,
 87871: 4,
 109681: 4,
 35762: 4,
 61325: 4,
 44105: 1,
 56905: 1,
 35644: 4,
 73347: 4,
 74405: 9,
 106903: 9,
 74801: 9,
 113245: 10,
 113425: 10,
 28569: 5,
 71280: 5,
 79312: 11,
 106340: 11,
 80242: 12,
 80610: 12,
 108195: 1,
 113710: 1,
 62623: 5,
 89245: 5,
 45539: 4,
 52373: 4,
 103313: 13,
 116023: 13,
 92378: 14,
 92380: 14,
 63711: 1,
 64855: 1,
 8210: 1,
 31952: 1,
 105355: 4,
 128178: 4,
 88962: 15,
 95361: 15,
 8695: 4,
 100641: 4,
 3790: 16,
 13385: 16,
 94865: 17,
 96483: 17,
 30766: 18,
 35428: 18,
 12377: 2,
 18747: 2,
 51754: 1,
 60937: 1,
 111917: 19,
 111940: 19,
 132682: 20,
 132708: 20,
 88020: 21,
 97322: 21,
 124562: 22,
 125277: 22,
 73506: 23,
 74976: 23,
 96336: 1,
 114706: 1,
 83006: 24,
 86

In [10]:
sample_frame['text_group'] = sample_frame['text_no'].apply(lambda x: id2component[x] if x in id2component else 'nf')

In [11]:
sample_frame.head()

,Unnamed: 0,date,title,description,link,postUrl,score,handle,platformId,id,message,caption,clean_text,textlen,text_no,text_group
0,3,2021-07-31 23:57:55,NaN,NaN,NaN,https://www.facebook.com/groups/20430641726032...,-1.296296,"*श्री ""राधा रानी"" प्राण हमारी ""वृंदावन"" मेरो ज...",2043064172603248,8043682,सती विरह में शिव घूमे लेकर देह तीनो लोक उमा पा...,NaN,सती विरह में शिव घूमे लेकर देह तीनो लोक उमा पा...,22,0,nf
1,4,2021-07-31 23:57:40,NaN,NaN,NaN,https://www.facebook.com/groups/33274896690677...,1.321429,Satsang-सत्संग-पौराणिक कहानियाँ-ध्यान-भक्ति-री...,332748966906779,9060585,सती विरह में शिव घूमे लेकर देह तीनो लोक उमा पा...,NaN,सती विरह में शिव घूमे लेकर देह तीनो लोक उमा पा...,22,1,nf
2,9,2021-07-31 23:54:58,मनकामना माईको दर्शन गरि आजको राशिफल हेर्नुहोस।...,वि.सं.२०७८ साल साउन १७ गते । आईतबार । ईश्वी सन...,https://khabarpatrikanp.com/archives/22725,https://www.facebook.com/groups/12800605888173...,-3.571429,राजा ज्ञानेन्द्र - King Gyanendra (राष्ट्रभक्त...,1280060588817309,14759682,मनकामना माईको दर्शन गरि आजको राशिफल हेर्नुहोस।...,khabarpatrikanp.com,मनकामना माईको दर्शन गरि आजको राशिफल हेर्नुहोस ...,17,2,nf
3,10,2021-07-31 23:54:54,"भगवती माताले सबैको मनोकांक्षा पुरा गरुन,श्रावण...","मेष राशि चु, चे, चो, ला, लि, लु, ले, लो, अ साम...",https://www.mbinfomedia.com/archives/3645,https://www.facebook.com/groups/54992145928912...,-1.780488,🇳🇵कृष्ण कडेल A.B.C इन्द्रेणी समर्थक G.C🇳🇵,549921459289121,14658604,"भगवती माताले सबैको मनोकांक्षा पुरा गरुन,श्रावण...",mbinfomedia.com,भगवती माताले सबैको मनोकांक्षा पुरा गरुनश्रावण ...,16,3,4
4,16,2021-07-31 23:52:03,"बिहार ,अवैध शराब के खिलाफ कार्यवाई में पुलिस क...",मगध एक्सप्रेस ;-बिहार के मुजफ्फरपुर पुलिस की व...,https://magadhexpress.in/?p=71252,https://www.facebook.com/groups/25065276533930...,-10.333333,औरंगाबाद जिले का नाम देव हो,250652765339303,9063079,"बिहार ,अवैध शराब के खिलाफ कार्यवाई में पुलिस क...",magadhexpress.in,बिहार अवैध शराब के खिलाफ कार्यवाई में पुलिस को...,27,4,nf


In [12]:
len(sample_frame), len(id2component.keys())

(136875, 27921)

In [13]:
frame = sample_frame.loc[sample_frame['text_group']!='nf']
len(frame)

27921

In [14]:
pd.set_option("max_colwidth", -1)
frame.loc[frame['text_group']==3][['postUrl','message']]#['postUrl'].tolist()

<ipython-input-14-4cd6234d6fb1>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("max_colwidth", -1)


,postUrl,message
31141,https://www.facebook.com/groups/387252001695575/permalink/1246958129058287,तेल के दाम कुछ कम कर दीजिये हर कोई आप की तरह बीबी और घर बार छोड़ कर तो नहीं भाग सकता हैं साहेब l
40311,https://www.facebook.com/groups/235972714704726/permalink/363021795333150,तेल के दाम कुछ कम कर दीजिये मोली साहब👉🎅 हर कोई आप की तरह बीबी और घर - बार छोड़ कर तो नहीं भाग सकता हैं 🤪🤪
43619,https://www.facebook.com/groups/2368559283453001/permalink/2572452809730313,तेल के दाम कुछ कम कर दीजिये साहब! हर कोई आप की तरह बीबी और घर-बार छोड़ कर तो नहीं भाग सकता हैं ।
46790,https://www.facebook.com/groups/309633546227737/permalink/1106315456559538,तेल के दाम कुछ कम कर दीजिये मोली साहब👉🎅 हर कोई आप की तरह बीबी और घर - बार छोड़ कर तो नहीं भाग सकता हैं 🤪🤪
47231,https://www.facebook.com/groups/193435544444640/permalink/1291387157982801,तेल के दाम कुछ कम कर दीजिये साहब! हर कोई आप की तरह बीबी और घर-बार छोड़ कर तो नहीं भाग सकता हैं ।
47801,https://www.facebook.com/groups/136875904551197/permalink/261472765424843,"तेल के दाम कुछ कम कर दीजिए, हर कोई आप की तरह बीवी और घर बार छोड़कर तो नहीं भाग सकता है साहेब....!😷"
49565,https://www.facebook.com/groups/637759342949631/permalink/4385042711554590,तेल के दाम कुछ कम कर दीजिये मोली साहब👉🎅 हर कोई आप की तरह बीबी और घर - बार छोड़ कर तो नहीं भाग सकता हैं 🤪🤪
49583,https://www.facebook.com/groups/2329165787363501/permalink/3041216662825073,तेल के दाम कुछ कम कर दीजिये मोली साहब👉🎅 हर कोई आप की तरह बीबी और घर - बार छोड़ कर तो नहीं भाग सकता हैं 🤪🤪
49585,https://www.facebook.com/groups/207480146741973/permalink/1006135243543122,तेल के दाम कुछ कम कर दीजिये मोली साहब👉🎅 हर कोई आप की तरह बीबी और घर - बार छोड़ कर तो नहीं भाग सकता हैं 🤪🤪
51585,https://www.facebook.com/groups/136875904551197/permalink/260963302142456,तेल के दाम कुछ कम कर दीजिये हर कोई आप की तरह बीबी और घर बार छोड़ कर तो नहीं भाग सकता हैं साहेब l 😗😗


In [15]:
frame.to_csv(writepath)